# CS224N Assignment 2: word2vec (44 Points)

## PARTE 1 Escrito (26 points)
Refresquemos rápidamente el algoritmo word2vec. La intuición clave detrás de word2vec es que ‘Se conoce a una palabra por la compañia que tiene’.
Concretamente, suponga que tenemos una palabra central $c$ y una ventana de contexto alrededor de $c$.

Nos referiremos a las palabras contenidas en la ventana de contexto como las 'palabras de afuera'

Por ejemplo, en la figura se observa que la palabra central $c$ es 'banking’. ya que el tamaño de la ventana de contexto es 2, las palabras de afuera son ‘turning’, ‘into’, ‘crises’, and ‘as’.
El objetivo del algoritmo word2vec skip-gram es el de aprender precisamente la distribución de probabilidad $P(O|C)$

Dada una palabra específica $o$ y una palabra específica $c$, queremos calcular $P(O=o|C=c)$, que es la probabilidad de que una palabra $o$ sea una palabra de afuera para $c$, ej. la probabilidad que $o$ caiga dentro de una ventana en el contexto de $c$

![word2vec](./imgs/word2vec_prob.png)

En word2vec, la distribución de probabilidad condicional es dada tomando el producto intero y aplicando la función softmax.

$$ P(O = o | C = c) = \frac{exp(u_o^T v_c)}{\sum_{w\in vocab}exp(u_w^T v_c)} $$ (1)

Aquí, $u_o$ es el vector de 'afuera' representando la palabra 'o' de afuera, y $v_c$ es el vector del 'centro' representando la palabra del centro $c$.
Para almacenar estos parámetros, definimos dos matrices, $U$ y $V$.
Las columnas de $U$ son todos los vectores $u_w$ de 'afuera'.
Las columnas de $V$ son todos los vectores $v_w$ del 'centro'.
Ambos $U$ y $V$ contienen un vector para cada $w\in Vocabulary$

Recordar de las clases que, para un par de palabras $c$ y $o$ la función de costo está dada por:

$$J_{naive-softmax}(v_c, o, U) = -log\ P(O=o|C=c) $$ (2)

Se puede ver este coste como la entropía cruzada entre la verdadera distribución de $y$ y la distribución estimada $\hat y$.

Recordar que la entropía cruzada entre una distribución de probabilidad verdadera $p$ y otra distribución $q$ para una variable discreta $x$ era:
$$ H(p,q) = -\sum_{i}p_i\ log(q_i) $$ <br>
Aquí, tanto $y$ como $\hat y$ son vectores de igual longitud al número de palabras del vocabulario. Más allá, la componente $k^{esima}$ de estos vectores indican la probabilidad condicional de que la $k^{esima}$ palabra sea una palabra de 'afuera' dada la palabra 'c'.
La verdadera distribución empírica de $y$ es un *one-hot vector* con un 1 para la verdadera palabra 'outside' y 0 en otro caso. La distribución estimada $\hat y$ es la distribución de probabilidad $P(O|C = c)$ Dado nuestro modelo de la ecuación (1).

## PARTE 1a (3 points)
Mostrar que la función de costo Naive-Softmax coincide con la función de coste de Cross-Entropy entre $y$ y $\hat{y}$

$$J=-\sum_{w\in Vocab} y_w\ log\ (\hat y_w) = -log(\hat y_w)$$

La respuesta debe ser una línea

<span style="color:green">

Respuesta

</span>

## PARTE 1b (5 points)
Calcular la derivada parcial de la función $J_{naive-softmax}(v_c, o, U)$ respecto a $v_c$.
Escribir la respuesta en términos de $y$, $\hat y$ y $U$.
<br><br><br>

·Se espera que las respuestas finales cumplan la **convención de dimensión**. Esto significa que la derivada parcial de cualquier función $f(x)$ respecto a $x$ debe tener la misma dimensión que $x$.

·Presentar la respuestas en su **forma vectorizada**. (No deberías referirte a elementos específicos de $y$, $\hat y$ o $U$ como $y_1, y_2, ...)

Esto nos permite minimizar eficientemente una función sin preocuparnos por redimensionar las matrices al utilizar el gradiente descendiente.
Utilizando esta convención, nos garantizamos que la regla de actualización
$θ := θ − α \frac{\partial J(θ)}{\partial \theta}$ esté bien definida.

<span style="color:green">

Respuesta

</span>

## PARTE 1c (5 points)
Calcular la derivada parcial de $J_{naive-softmax}(v_c, o, U)$ respecto a cada word vector de salida $u_w$. <br>
Habrá dos casos: cuando $w=o$ sea el verdadero vector de salida
y cuando $w\neq o$ para el resto de palabras

·Escriba su respuesta en términos de $y$, $\hat y$ y $v_c$. En esta parte podrás utilizar elementos específicos como $y_1$, $y_2$, etc.

<span style="color:green">

Respuesta

</span>

## PARTE 1d (1 point)
Calcular la derivadad parcial de $J_{naive-softmax}(v_c,o,U)$ respecto a $U$

·Escribir su respuesta en términos de $\frac{\partial J(v_c, o, U)}{\partial u_1}$, $\frac{\partial J(v_c, o, U)}{\partial u_2}$, ..., $\frac{\partial J(v_c, o, U)}{\partial u_{|Vocab|}}$

·La solución debe tener una o dos líneas

<span style="color:green">

Respuesta

</span>

## PARTE 1e (3 Points)
La función sigmoidea es<br>
$$σ(x) = \frac{1}{1 + e^{−x}} = \frac{e^x}{e^x + 1} $$<br>
Obtener la derivada de $σ(x)$ respecto a $x$, donde $x$ es un escalar.<br>
Pista: Querrás reescribir tu respuesta en términos de $σ(x)$.

<span style="color:green">

Respuesta

</span>

## PARTE 1f (4 points)
Ahora consideraremos la función de coste de **Negative Sampling** que es una alternativa a la función de coste Naive softmax.<br>
Suponer que se toman del vocabulario $K$ muestras negativas (palabras).
Por sencillez de notación, nos referiremos a ellas por $w_1, w_2, ..., w_K$
Y a sus vectores de salida como $u_1, ..., u_K$.

Para esta pregunta, suponer que las $K$ muestras son diferentes, en otras palabras, $w_i\neq w_j$ para $i\neq j, i,j\in\{1,...,K\}$

·Observar que para $o\neq\{w_1, ..., w_K\}$. Para una palabra central $c$ y una palabra externa $o$, la función de costo de Negative-Sampling es

$$ J_{neg-sample}(v_c, o, U) = -log(σ(u_o^T v_c))−\sum_{k=1}^K log(σ(−u_k^T v_c)) $$

* Repetir los ejercicios 1b y 1c, calculando las derivadas parciales de $J_{neg-sample}$ respecto a $v_c$, respecto a $u_o$ y respecto la muestra negativa $u_k$.<br>
* Escribir las respuestas en términos de vectores de $u_o$, $v_c$ y $u_k$ donde $k\in [1,K]$

* Al finalizar, escribir en una oración, por qué esta función de coste es mucho más eficiente de calcular que la función Naive-Softmax.

·Obs. Debes ser capaz de utilizar la solución del 1e para calcular los gradientes requeridos.

<span style="color:green">

Respuesta

</span>


## PARTE 1g (2 point)

Repetiremos el ejercicio anterior, pero sin suponer que las $K$ muestras eran diferentes. Suponer que las $K$ muestras se tomaron del vocabulario.<br>
·Por sencillez de notación, nos referiremos como $w_1, w_2,...,w_K$ y a sus vectores de salida como $u_1, ..., u_K$

En esta pregunta, no puedes suponer que las palabras son diferentes, en otras palabras, $w_i=w_j$ podría ser cierto cuando $i=j$ lo sea. Notar que $o\notin\{w_1,...,w_K\}$. Para una palabra central $c$ una palabra de salida $o$, la función de coste Negative-Sampling está dada por:<br>

$$ J_{neg-sample}(v_c, o, U) = -log(σ(u_o^T v_c))−\sum_{k=1}^K log(σ(−u_k^T v_c)) $$

* Calcular la derivada parcial de $J_{neg-sample}$ respecto a la muestra negativa $u_k$.<br>
* Escribir las respuestas en términos de vecotres $v_c$ y $u_k$, donde $k\in[1,K]$.

Pista: Partir la sumatoria de la función de costo en dos partes, la suma sobre todas las palabras muestreadas igual a $u_k$ y la suma sobre todas las palabras muestreadas distintas de $u_k$.

<span style="color:green">

Respuesta

</span>

## PARTE 1h (3 points)
Suponga que la palabra central es $c=w_t$ y la ventana de contexto es
$[w_{t−m}, ..., w_{t−1}, w_t, w_{t+1}, ..., w_{t+m}]$, donde $m$ es el tamaño de la ventana. Recordar que para la versión skip-gram de word2vec, el costo total para la ventana de contexto es
$$ J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U) = \sum_{−m≤j≤m\atop j\neq 0} J(v_c, w_{t+j}, U) $$

Donde $J(v_c, w_{t+j}, U)$ representa un coste arbitrario en términos de la palabra central $c=w_t$ y la palabra de salida $w{t+j}$.
 $J(v_c, w_{t+j}, U)$ podría ser $J_{naive-softmax}(v_c, w_{t+j}, U)$ o $J_{neg-sample}(v_c, w_{t+j}, U), dependiendo de tu implementación.

Escribir las 3 derivadas parciales:
1. $\frac{\partial}{\partial U}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)$
3. $\frac{\partial}{\partial v_c}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)$
3. $\frac{\partial}{\partial v_w}J_{skip-gram}(v_c, w_{t−m}, ..., w_{t+m}, U)$ cuando $w\neq c$

·Escribir las respuestas en términos de $\frac{\partial}{\partial U}J(v_c, w_{t+j}, U)$ y $\frac{\partial}{\partial v_c}J(v_c, w_{t+j}, U)$.
Esto es muy sencillo, cada solución debe ser una línea.

<span style="color:green">

Respuesta

</span>

# PARTE 2 Código (18 points)
En esta parte implementarás el modelo word2vec y entrenarás tus propios word vectors mediante Stochastic Gradient Descent (SGD).
Antes de comenzar, se deben ejecutar las siguientes instrucciones dentro del directorio del proyecto con el fin de crear el apropiado entorno virtual para conda.
 Esto garantiza que tienes todos las librerías necesarias para completar esta parte.
·Obs. Debes terminar la sección matemática anterior antes de intentar implementar el código ya que se pedirá implementar estas funciones matemáticas.
También querrás implementar y testear la siguientes secciones en orden ya que son acumulativas.

` conda env create -f env.yml `<br>
` conda activate a2 `

Al terminar con el ejercicio, puedes desactivas este entorno con <br>
` conda deactivate `<br>

Para cada método, en los comentarios hemos incluído aproximadamente la cantidad de líneas que nuestra solución tiene para que tomes de referencia.
 Los bucles for en python toman más tiempo en finalizar cuando se utilizan sobre arrays grandes, por eso esperamos que utilices los métodos de numpy. Estaremos controlando la eficiencia de tu código, serás capaz de ver los resultados del autograder cuando cargues el código al Gradescope. Se recomienda publicarlo temprana y continuamente.

In [ ]:
#Imports

## PARTE 2a  (12 points)
Empezaremos implementando los métodos dentro de **word2vec.py**

Puedes testear un método $m$ particular ejecutando
` python word2vec.py m `<br>
Por ejemplo, puedes testear la función sigmoidea ejecutando: <br>
` python word2vec.py sigmoid `

Ejercicios:
1. Implementar el método **sigmoid**, que recive un vector y le aplica la función sigmoidea.
2. Implementar la función de coste softmax y gradiente en el método **naiveSoftmaxLossAndGradient**
3. Implement the negative sampling loss and gradient in the negSamplingLossAndGradient
method.
4. Implementar el modelo skip-gram en el método **skipgram**.
Al finalizar, verificar la implementación entera ejecutando <br>
` python word2vec.py `

In [ ]:
#code 2a

## PARTE 2b  (4 points)
Complete la implementación para el optimizador SGD en el método **sgd.py**. Verificar tu implementación ejecutando <br>
` python sgd.py `

In [ ]:
#code 2b

## PARTE 2c (2 points)
Hora de Exposición! Es la hora de cargar datos reales y entrenar los word vectors con todo ya implementado! Utilizaremos el DataSet **Stanford Sentiment Treebank (SST)** para entrenar los word vectors, y luego aplicarlos en una simple tarea de análisis del sentimiento.<br>

Para obtener el dataset, ejecutar:<br>
` sh get datasets.sh `<br>
 No hay código adicional para escribir en esta parte, solo ejecutar: <br>
` python run.py `

Obs.: El proceso de entrenamiento podría llevar mucho tiempo dependiendo de la eficiencia de tu implementación y el poder de cálculo de tu computadora (a una implementación eficiente le lleva entre 1 y 2 horas). Planificar acorde!

Luego de 40,000 iteraciones, el script terminará y aparecerá un gráfico para tus word vectors.
Además, se guardará como **word_vectors.png** en tu directorio. Incluir el gráfico en tu documento escrito, explicando brevemente al menos 3 oraciones que veas en el gráfico.

In [ ]:
#code 2c